## Open the text files with the entries :
***


In [29]:
import os 

path_db = "/home/robbyce/Documents/bioinformatics/Depolymerase_DB"
#path_db = "/home/conchae/databases/depolymerase_building"

depo_list = open(f"{path_db}/IPR_depolymerase.16012023.txt").read().split("\n")[0:-1]
possible_depo_list = open(f"{path_db}/IPR_possible_depo.16012023.txt").read().split("\n")[0:-1]

IPR_depo = depo_list + possible_depo_list

urls = [f"https://www.ebi.ac.uk:443/interpro/api/protein/UniProt/entry/InterPro/{IPR}/?page_size=200&extra_fields=sequence" for IPR in depo_list if IPR]


In [ ]:
# standard library modules
import sys, errno, re, json, ssl
from urllib import request
from urllib.error import HTTPError
from time import sleep
import os 
from multiprocessing import Pool

path_db = "/home/conchae/databases/depolymerase_building"

HEADER_SEPARATOR = "|"
LINE_LENGTH = 80


IPR_depo = [
 'IPR029411', 'IPR002152', 'IPR012480', 'IPR039513', 'IPR002925', 'IPR041624', 'IPR024200', 'IPR029456', 'IPR015220', 'IPR041111', 'IPR036962',
 'IPR016285', 'IPR005323', 'IPR016007', 'IPR044505', 'IPR015178', 'IPR011840', 'IPR024561', 'IPR006048', 'IPR009470', 'IPR004197', 'IPR012878',
 'IPR015165', 'IPR024732', 'IPR023309', 'IPR040784', 'IPR014551', 'IPR032312', 'IPR000490', 'IPR006103', 'IPR001139', 'IPR031330', 'IPR000400',
 'IPR044846', 'IPR011040', 'IPR011050', 'IPR024428', 'IPR000743', 'IPR002241', 'IPR005192', 'IPR011837', 'IPR024430', 'IPR000852', 'IPR006710',
 'IPR011839', 'IPR010905', 'IPR013775', 'IPR011838', 'IPR026071', 'IPR032792', 'IPR043534', 'IPR012939', 'IPR037110', 'IPR003469', 'IPR017736',
 'IPR015289', 'IPR016283', 'IPR016282', 'IPR039743', 'IPR005197', 'IPR016289', 'IPR018155', 'IPR035669', 'IPR023933', 'IPR036026', 'IPR023296',
 'IPR016840', 'IPR013777', 'IPR000165', 'IPR010720', 'IPR001554', 'IPR001661', 'IPR002252', 'IPR027291', 'IPR006215', 'IPR005604', 'IPR019563',
 'IPR022844', 'IPR011395', 'IPR011496', 'IPR015883', 'IPR005198', 'IPR017069', 'IPR022859', 'IPR025975', 'IPR018082', 'IPR012334', 'IPR000322',
 'IPR001860', 'IPR000805', 'IPR036434', 'IPR038964', 'IPR006046', 'IPR024745', 'IPR013319', 'IPR000334', 'IPR031335', 'IPR003476', 'IPR001382',
 'IPR037019', 'IPR005193', 'IPR037398', 'IPR013785', 'IPR010702', 'IPR026856', 'IPR023295', 'IPR039174', 'IPR034641', 'IPR036278', 'IPR045032',
 'IPR016590', 'IPR002022', 'IPR012970', 'IPR038970', 'IPR044112', 'IPR000514', 'IPR001724', 'IPR001362', 'IPR025092', 'IPR006775', 'IPR036881',
 'IPR033654', 'IPR039279', 'IPR006047', 'IPR006101', 'IPR016288', 'IPR001137', 'IPR004185', 'IPR004888', 'IPR001547', 'IPR005199', 'IPR011583',
 'IPR000933', 'IPR011683', 'IPR001722', 'IPR005201', 'IPR001439', 'IPR002037', 'IPR011100', 'IPR027260', 'IPR045857', 'IPR009860', 'IPR016455',
 'IPR014895', 'IPR027946', 'IPR008929', 'IPR000125', 'IPR024746', 'IPR001329', 'IPR000726', 'IPR000974', 'IPR011330', 'IPR006633', 'IPR008979',
 'IPR023346', 'IPR000974', 'IPR019282', 'IPR000922', 'IPR041351', 'IPR006421', 'IPR003790', 'IPR001088', 'IPR010713', 'IPR023309', 'IPR011839',
 'IPR000125', 'IPR035394', 'IPR032091', 'IPR015177', 'IPR039448', 'IPR015331', 'IPR011613', 'IPR015179', 'IPR014635', 'IPR001371', 'IPR016282',
 'IPR016714', 'IPR006065', 'IPR001223', 'IPR025706', 'IPR041542', 'IPR033452', 'IPR014718', 'IPR035396', 'IPR044914', 'IPR006425', 'IPR021016',
 'IPR008811', 'IPR001286', 'IPR005200', 'IPR024733', 'IPR000757', 'IPR032790', 'IPR008397', 'IPR004898', 'IPR006626', 'IPR003159', 'IPR008902',
 'IPR007724', 'IPR046372', 'IPR001000', 'IPR016287', 'IPR013776', 'IPR002594', 'IPR023720', 'IPR000556', 'IPR016286', 'IPR016828', 'IPR029070',
 'IPR006584', 'IPR039514', 'IPR026283', 'IPR032979', 'IPR009939', 'IPR039473', 'IPR007781', 'IPR000677', 'IPR038901', 'IPR008291', 'IPR040527',
 'IPR036439']

urls = [f"https://www.ebi.ac.uk:443/interpro/api/protein/UniProt/entry/InterPro/{IPR}/?page_size=200&extra_fields=sequence" for IPR in IPR_depo if IPR]

def get_IPR(url) :
    IPR = url.split("InterPro/")[1].split("/")[0]
    print(f"{IPR} in the process... \n")
    final_length = 0
    if os.path.isfile(f"{path_db}/{IPR}.entry.sequences.fasta") == False :
        with open(f"{path_db}/{IPR}.entry.sequences.fasta", "w") as outfile :
            #disable SSL verification to avoid config issues
            context = ssl._create_unverified_context()
            next = url
            last_page = False
            attempts = 0
            while next:
                try:
                    req = request.Request(next, headers={"Accept": "application/json"})
                    res = request.urlopen(req, context=context)
                    # If the API times out due a long running query
                    if res.status == 408:
                        # wait just over a minute
                        sleep(61)
                        # then continue this loop with the same URL
                        continue
                    elif res.status == 204:
                        #no data so leave loop
                        break
                    payload = json.loads(res.read().decode())
                    next = payload["next"]
                    attempts = 0
                    if not next:
                        last_page = True
                except HTTPError as e:
                    if e.code == 408:
                        sleep(61)
                        continue
                    else:
                        # If there is a different HTTP error, it wil re-try 3 times before failing
                        if attempts < 3:
                            attempts += 1
                            sleep(61)
                            continue
                        else:
                            sys.stderr.write("LAST URL: " + next)
                            raise e

                for i, item in enumerate(payload["results"]):
                    if i > 500 :
                    start , end = 0, 0

                    entries = None
                    if ("entry_subset" in item):
                        entries = item["entry_subset"]
                    elif ("entries" in item):
                        entries = item["entries"]
                    if entries is not None:
                        entries_header = "-".join([entry["accession"] + "(" + ";".join([",".join([ str(fragment["start"]) + "..." + str(fragment["end"]) 
                              for fragment in locations["fragments"]]) for locations in entry["entry_protein_locations"]]) + ")" for entry in entries])
                        outfile.write(">" + item["metadata"]["accession"] + HEADER_SEPARATOR
                                      + entries_header + HEADER_SEPARATOR
                                      + item["metadata"]["name"] + "\n")
                    else:
                        outfile.write(">" + item["metadata"]["accession"] + HEADER_SEPARATOR + item["metadata"]["name"] + "\n")
                        pass
                    seq = item["extra_fields"]["sequence"]
                    fastaSeqFragments = [seq[0+i:LINE_LENGTH+i] for i in range(0, len(seq), LINE_LENGTH)]
                    for fastaSeqFragment in fastaSeqFragments:
                        outfile.write(fastaSeqFragment + "\n")
                    # Don't overload the server, give it time before asking for more
                if next:
                    sleep(1)
                    
if __name__ == "__main__" :
    with Pool(20) as p :
        p.map(get_IPR , urls)

In [30]:
import pprint 
pp = pprint.PrettyPrinter(width = 150, compact = True)

pp.pprint(IPR_depo)

['IPR029411', 'IPR002152', 'IPR012480', 'IPR039513', 'IPR002925', 'IPR041624', 'IPR024200', 'IPR029456', 'IPR015220', 'IPR041111', 'IPR036962',
 'IPR016285', 'IPR005323', 'IPR016007', 'IPR044505', 'IPR015178', 'IPR011840', 'IPR024561', 'IPR006048', 'IPR009470', 'IPR004197', 'IPR012878',
 'IPR015165', 'IPR024732', 'IPR023309', 'IPR040784', 'IPR014551', 'IPR032312', 'IPR000490', 'IPR006103', 'IPR001139', 'IPR031330', 'IPR000400',
 'IPR044846', 'IPR011040', 'IPR011050', 'IPR024428', 'IPR000743', 'IPR002241', 'IPR005192', 'IPR011837', 'IPR024430', 'IPR000852', 'IPR006710',
 'IPR011839', 'IPR010905', 'IPR013775', 'IPR011838', 'IPR026071', 'IPR032792', 'IPR043534', 'IPR012939', 'IPR037110', 'IPR003469', 'IPR017736',
 'IPR015289', 'IPR016283', 'IPR016282', 'IPR039743', 'IPR005197', 'IPR016289', 'IPR018155', 'IPR035669', 'IPR023933', 'IPR036026', 'IPR023296',
 'IPR016840', 'IPR013777', 'IPR000165', 'IPR010720', 'IPR001554', 'IPR001661', 'IPR002252', 'IPR027291', 'IPR006215', 'IPR005604', 'IPR0

In [13]:
#!/usr/bin/env python3

# standard library modules
import sys, errno, re, json, ssl
from urllib import request
from urllib.error import HTTPError
from time import sleep


HEADER_SEPARATOR = "|"
LINE_LENGTH = 80
BASE_URL = f"https://www.ebi.ac.uk:443/interpro/api/protein/UniProt/entry/InterPro/IPR041351/?page_size=200&extra_fields=sequence"
urls = [f"https://www.ebi.ac.uk:443/interpro/api/protein/UniProt/entry/InterPro/{IPR}/?page_size=200&extra_fields=sequence" for IPR in depo_list if IPR]

def output_list():
    #disable SSL verification to avoid config issues
    context = ssl._create_unverified_context()
    next = BASE_URL
    last_page = False
    attempts = 0
    while next:
        try:
            req = request.Request(next, headers={"Accept": "application/json"})
            res = request.urlopen(req, context=context)
            # If the API times out due a long running query
            if res.status == 408:
                # wait just over a minute
                sleep(61)
                # then continue this loop with the same URL
                continue
            elif res.status == 204:
                #no data so leave loop
                break
            payload = json.loads(res.read().decode())
            next = payload["next"]
            attempts = 0
            if not next:
                last_page = True
        except HTTPError as e:
            if e.code == 408:
                sleep(61)
                continue
            else:
                # If there is a different HTTP error, it wil re-try 3 times before failing
                if attempts < 3:
                    attempts += 1
                    sleep(61)
                    continue
                else:
                    sys.stderr.write("LAST URL: " + next)
                    raise e

        for i, item in enumerate(payload["results"]):
            start , end = 0, 0

            entries = None
            if ("entry_subset" in item):
                entries = item["entry_subset"]
            elif ("entries" in item):
                entries = item["entries"]

            if entries is not None:
                entries_header = "-".join([entry["accession"] + "(" + ";".join([",".join([ str(fragment["start"]) + "..." + str(fragment["end"]) 
                      for fragment in locations["fragments"]]) for locations in entry["entry_protein_locations"]]) + ")" for entry in entries])
                #start = fragment["start"]
                #end = fragment["end"]
                sys.stdout.write(">" + item["metadata"]["accession"] + HEADER_SEPARATOR
                              + entries_header + HEADER_SEPARATOR
                              + item["metadata"]["name"] + "\n")
                start = int(entries_header.split("...")[0].split("(")[1])
                end = int(entries_header.split("...")[1].split(")")[0])
            else:
                sys.stdout.write(">" + item["metadata"]["accession"] + HEADER_SEPARATOR + item["metadata"]["name"] + "\n")
                pass

            seq = item["extra_fields"]["sequence"]
            fastaSeqFragments = [seq[0+i:LINE_LENGTH+i] for i in range(0, len(seq), LINE_LENGTH)]
            #fasta_domain = "".join(fastaSeqFragments)[start:end]
            #sys.stdout.write(fasta_domain + "\n")
            for fastaSeqFragment in fastaSeqFragments:
                sys.stdout.write(fastaSeqFragment + "\n")
            # Don't overload the server, give it time before asking for more
        if next:
            sleep(1)

if __name__ == "__main__":
    output_list()

URLError: <urlopen error unknown url type: ['https>

In [49]:
#!/usr/bin/env python3

# standard library modules
import sys, errno, re, json, ssl
from urllib import request
from urllib.error import HTTPError
from time import sleep


HEADER_SEPARATOR = "|"
LINE_LENGTH = 80
urls = [f"https://www.ebi.ac.uk:443/interpro/api/protein/UniProt/entry/InterPro/{IPR}/?page_size=200&extra_fields=sequence" for IPR in depo_list if IPR]

def output_list(BASE_URL):
    #disable SSL verification to avoid config issues
    context = ssl._create_unverified_context()
    next = BASE_URL
    last_page = False
    attempts = 0
    print("Launching")
    while next:
        try:
            req = request.Request(next, headers={"Accept": "application/json"})
            res = request.urlopen(req, context=context)
            # If the API times out due a long running query
            if res.status == 408:
                # wait just over a minute
                sleep(61)
                # then continue this loop with the same URL
                continue
            elif res.status == 204:
                #no data so leave loop
                break
            payload = json.loads(res.read().decode())
            next = payload["next"]
            attempts = 0
            if not next:
                last_page = True
        except HTTPError as e:
            if e.code == 408:
                sleep(61)
                continue
            else:
                # If there is a different HTTP error, it wil re-try 3 times before failing
                if attempts < 3:
                    attempts += 1
                    sleep(61)
                    continue
                else:
                    sys.stderr.write("LAST URL: " + next)
                    raise e
        for i, item in enumerate(payload["results"]):
            start , end = 0, 0
            entries = None
            if ("entry_subset" in item):
                entries = item["entry_subset"]
            elif ("entries" in item):
                entries = item["entries"]
            if entries is not None:
                entries_header = "-".join([entry["accession"] + "(" + ";".join([",".join([ str(fragment["start"]) + "..." + str(fragment["end"]) 
                      for fragment in locations["fragments"]]) for locations in entry["entry_protein_locations"]]) + ")" for entry in entries])
                #start = fragment["start"]
                #end = fragment["end"]
                sys.stdout.write(">" + item["metadata"]["accession"] + HEADER_SEPARATOR
                              + entries_header + HEADER_SEPARATOR
                              + item["metadata"]["name"] + "\n")
                start = int(entries_header.split("...")[0].split("(")[1])
                end = int(entries_header.split("...")[1].split(")")[0])
            else:
                return (">" + item["metadata"]["accession"] + HEADER_SEPARATOR + item["metadata"]["name"] + "\n")
                pass
            seq = item["extra_fields"]["sequence"]
            fastaSeqFragments = [seq[0+i:LINE_LENGTH+i] for i in range(0, len(seq), LINE_LENGTH)]
            #fasta_domain = "".join(fastaSeqFragments)[start:end]
            #sys.stdout.write(fasta_domain + "\n")
            for fastaSeqFragment in fastaSeqFragments:
                return(fastaSeqFragment + "\n")
            # Don't overload the server, give it time before asking for more
        if next:
            sleep(1)

#if __name__ == "__main__":
resulst = map(output_list , urls)

In [56]:
#!/usr/bin/env python3

# standard library modules
import sys, errno, re, json, ssl
from urllib import request
from urllib.error import HTTPError
from time import sleep


HEADER_SEPARATOR = "|"
LINE_LENGTH = 80
BASE_URL = f"https://www.ebi.ac.uk:443/interpro/api/protein/UniProt/entry/InterPro/IPR041351/?page_size=200&extra_fields=sequence"
urls = [f"https://www.ebi.ac.uk:443/interpro/api/protein/UniProt/entry/InterPro/{IPR}/?page_size=200&extra_fields=sequence" for IPR in depo_list if IPR]

for url in urls :
    IPR = url.split("InterPro/")[1].split("/")[0]
    print(f"{IPR} in the process... \n")
    final_length = 0
    with open(f"{path_db}/{IPR}.entry.sequences.fasta", "w") as outfile :
        #disable SSL verification to avoid config issues
        context = ssl._create_unverified_context()
        next = url
        last_page = False
        attempts = 0
        while next:
            try:
                req = request.Request(next, headers={"Accept": "application/json"})
                res = request.urlopen(req, context=context)
                # If the API times out due a long running query
                if res.status == 408:
                    # wait just over a minute
                    sleep(61)
                    # then continue this loop with the same URL
                    continue
                elif res.status == 204:
                    #no data so leave loop
                    break
                payload = json.loads(res.read().decode())
                next = payload["next"]
                attempts = 0
                if not next:
                    last_page = True
            except HTTPError as e:
                if e.code == 408:
                    sleep(61)
                    continue
                else:
                    # If there is a different HTTP error, it wil re-try 3 times before failing
                    if attempts < 3:
                        attempts += 1
                        sleep(61)
                        continue
                    else:
                        sys.stderr.write("LAST URL: " + next)
                        raise e

            for i, item in enumerate(payload["results"]):
                if i > 500 :
                start , end = 0, 0

                entries = None
                if ("entry_subset" in item):
                    entries = item["entry_subset"]
                elif ("entries" in item):
                    entries = item["entries"]
                if entries is not None:
                    entries_header = "-".join([entry["accession"] + "(" + ";".join([",".join([ str(fragment["start"]) + "..." + str(fragment["end"]) 
                          for fragment in locations["fragments"]]) for locations in entry["entry_protein_locations"]]) + ")" for entry in entries])
                    outfile.write(">" + item["metadata"]["accession"] + HEADER_SEPARATOR
                                  + entries_header + HEADER_SEPARATOR
                                  + item["metadata"]["name"] + "\n")
                else:
                    outfile.write(">" + item["metadata"]["accession"] + HEADER_SEPARATOR + item["metadata"]["name"] + "\n")
                    pass
                seq = item["extra_fields"]["sequence"]
                fastaSeqFragments = [seq[0+i:LINE_LENGTH+i] for i in range(0, len(seq), LINE_LENGTH)]
                for fastaSeqFragment in fastaSeqFragments:
                    outfile.write(fastaSeqFragment + "\n")
                # Don't overload the server, give it time before asking for more
            if next:
                sleep(1)


IPR029411 in the process... 

IPR002152 in the process... 

IPR012480 in the process... 

IPR039513 in the process... 

IPR002925 in the process... 

IPR041624 in the process... 

IPR024200 in the process... 

IPR029456 in the process... 

IPR015220 in the process... 

IPR041111 in the process... 

IPR036962 in the process... 

IPR016285 in the process... 

IPR005323 in the process... 

IPR016007 in the process... 

IPR044505 in the process... 

IPR015178 in the process... 

IPR011840 in the process... 

IPR024561 in the process... 

IPR006048 in the process... 

IPR009470 in the process... 

IPR004197 in the process... 

IPR012878 in the process... 

IPR015165 in the process... 

IPR024732 in the process... 

IPR023309 in the process... 

IPR040784 in the process... 

IPR014551 in the process... 

IPR032312 in the process... 

IPR000490 in the process... 

IPR006103 in the process... 

IPR001139 in the process... 

IPR031330 in the process... 

IPR000400 in the process... 

IPR044846 

RemoteDisconnected: Remote end closed connection without response

In [ ]:
#!/bin/bash
#BATCH --job-name=IPR_download
#SBATCH --partition=medium
#SBATCH --ntasks=1 
#SBATCH --cpus-per-task=5
#SBATCH --mem=10gb 
#SBATCH --time=4-00:00:00 
#SBATCH --output=IPR_download%j.log 

source /storage/apps/ANACONDA/anaconda3/etc/profile.d/conda.sh
conda activate python_311

python /home/conchae/databases/depolymerase_building/download_IPR.py

In [ ]:
import sys, errno, re, json, ssl
from urllib import request
from urllib.error import HTTPError
from time import sleep


HEADER_SEPARATOR = "|"
LINE_LENGTH = 80
BASE_URL = f"https://www.ebi.ac.uk:443/interpro/api/protein/UniProt/entry/InterPro/IPR041351/?page_size=200&extra_fields=sequence"
#urls = [f"https://www.ebi.ac.uk:443/interpro/api/protein/UniProt/entry/InterPro/{IPR}/?page_size=200&extra_fields=sequence" for IPR in depo_list if IPR]

urls.reverse()

for url in urls :
    IPR = url.split("InterPro/")[1].split("/")[0]
    print(f"{IPR} in the process... \n")
    final_length = 0
    if os.path.isfile(f"{path_db}/{IPR}.entry.sequences.fasta") == False :
        with open(f"{path_db}/{IPR}.entry.sequences.fasta", "w") as outfile :
            #disable SSL verification to avoid config issues
            context = ssl._create_unverified_context()
            next = url
            last_page = False
            attempts = 0
            while next:
                try:
                    req = request.Request(next, headers={"Accept": "application/json"})
                    res = request.urlopen(req, context=context)
                    # If the API times out due a long running query
                    if res.status == 408:
                        # wait just over a minute
                        sleep(61)
                        # then continue this loop with the same URL
                        continue
                    elif res.status == 204:
                        #no data so leave loop
                        break
                    payload = json.loads(res.read().decode())
                    next = payload["next"]
                    attempts = 0
                    if not next:
                        last_page = True
                except HTTPError as e:
                    if e.code == 408:
                        sleep(61)
                        continue
                    else:
                        # If there is a different HTTP error, it wil re-try 3 times before failing
                        if attempts < 3:
                            attempts += 1
                            sleep(61)
                            continue
                        else:
                            sys.stderr.write("LAST URL: " + next)
                            raise e
                for i, item in enumerate(payload["results"]):
                    entries = None
                    if ("entry_subset" in item):
                        entries = item["entry_subset"]
                    elif ("entries" in item):
                        entries = item["entries"]
                    if entries is not None:
                        entries_header = "-".join([entry["accession"] + "(" + ";".join([",".join([ str(fragment["start"]) + "..." + str(fragment["end"]) 
                              for fragment in locations["fragments"]]) for locations in entry["entry_protein_locations"]]) + ")" for entry in entries])
                        outfile.write(">" + item["metadata"]["accession"] + HEADER_SEPARATOR
                                      + entries_header + HEADER_SEPARATOR
                                      + item["metadata"]["name"] + "\n")
                    else:
                        outfile.write(">" + item["metadata"]["accession"] + HEADER_SEPARATOR + item["metadata"]["name"] + "\n")
                        pass
                    seq = item["extra_fields"]["sequence"]
                    fastaSeqFragments = [seq[0+i:LINE_LENGTH+i] for i in range(0, len(seq), LINE_LENGTH)]
                    for fastaSeqFragment in fastaSeqFragments:
                        outfile.write(fastaSeqFragment + "\n")
                    # Don't overload the server, give it time before asking for more
                if next:
                    sleep(1)
    else :
        continue


***
# Completed 

***
# Formatting the database correctly :


In [2]:
type_dico = {
 'Domain': ['IPR029411', 'IPR002925', 'IPR041624', 'IPR029456', 'IPR015220', 'IPR041111',
            'IPR005323', 'IPR044505', 'IPR015178', 'IPR011840', 'IPR024561', 'IPR006048',
            'IPR009470', 'IPR004197', 'IPR015165', 'IPR024732', 'IPR040784', 'IPR032312',
            'IPR006103', 'IPR031330', 'IPR011040', 'IPR024428', 'IPR005192', 'IPR024430',
            'IPR011839', 'IPR011838', 'IPR032792', 'IPR012939', 'IPR015289', 'IPR010720',
            'IPR019563', 'IPR015883', 'IPR024745', 'IPR000334', 'IPR031335', 'IPR002022',
            'IPR012970', 'IPR006775', 'IPR033654', 'IPR006047', 'IPR004185', 'IPR001547',
            'IPR011583', 'IPR005201', 'IPR011100', 'IPR014895', 'IPR027946', 'IPR000726',
            'IPR006633', 'IPR019282', 'IPR000922', 'IPR041351', 'IPR003790', 'IPR010713',
            'IPR011839', 'IPR035394', 'IPR032091', 'IPR015177', 'IPR039448', 'IPR015331',
            'IPR011613', 'IPR015179', 'IPR001223', 'IPR025706', 'IPR041542', 'IPR033452',
            'IPR035396', 'IPR024733', 'IPR000757', 'IPR032790', 'IPR008397', 'IPR003159',
            'IPR008902', 'IPR046372', 'IPR001000', 'IPR006584', 'IPR039514', 'IPR039473',
            'IPR040527'],
 'Family': ['IPR002152', 'IPR012480', 'IPR039513', 'IPR024200', 'IPR016285', 'IPR016007',
            'IPR012878', 'IPR014551', 'IPR000490', 'IPR001139', 'IPR000400', 'IPR044846',
            'IPR000743', 'IPR002241', 'IPR011837', 'IPR000852', 'IPR006710', 'IPR010905',
            'IPR013775', 'IPR026071', 'IPR043534', 'IPR003469', 'IPR017736', 'IPR016283',
            'IPR016282', 'IPR039743', 'IPR005197', 'IPR016289', 'IPR018155', 'IPR035669',
            'IPR023933', 'IPR016840', 'IPR013777', 'IPR000165', 'IPR001554', 'IPR001661',
            'IPR002252', 'IPR006215', 'IPR005604', 'IPR022844', 'IPR011395', 'IPR011496',
            'IPR005198', 'IPR017069', 'IPR022859', 'IPR025975', 'IPR018082', 'IPR000322',
            'IPR001860', 'IPR000805', 'IPR038964', 'IPR006046', 'IPR003476', 'IPR001382',
            'IPR005193', 'IPR037398', 'IPR010702', 'IPR026856', 'IPR039174', 'IPR034641',
            'IPR045032', 'IPR016590', 'IPR038970', 'IPR044112', 'IPR000514', 'IPR001724',
            'IPR001362', 'IPR025092', 'IPR039279', 'IPR006101', 'IPR016288', 'IPR001137',
            'IPR004888', 'IPR005199', 'IPR000933', 'IPR011683', 'IPR001722', 'IPR001439',
            'IPR002037', 'IPR027260', 'IPR009860', 'IPR016455', 'IPR000125', 'IPR024746',
            'IPR001329', 'IPR000974', 'IPR000974', 'IPR006421', 'IPR001088', 'IPR000125',
            'IPR014635', 'IPR001371', 'IPR016282', 'IPR016714', 'IPR006065', 'IPR006425',
            'IPR021016', 'IPR008811', 'IPR001286', 'IPR005200', 'IPR004898', 'IPR007724',
            'IPR016287', 'IPR013776', 'IPR002594', 'IPR023720', 'IPR000556', 'IPR016286',
            'IPR016828', 'IPR026283', 'IPR032979', 'IPR009939', 'IPR007781', 'IPR000677',
            'IPR038901', 'IPR008291'],
 'Homologous_superfamily': ['IPR036962', 'IPR023309', 'IPR011050', 'IPR037110', 'IPR036026',
                            'IPR023296', 'IPR027291', 'IPR012334', 'IPR036434', 'IPR013319',
                            'IPR037019', 'IPR013785', 'IPR023295', 'IPR036278', 'IPR036881',
                            'IPR045857', 'IPR008929', 'IPR011330', 'IPR008979', 'IPR023346',
                            'IPR023309', 'IPR014718', 'IPR044914', 'IPR029070', 'IPR036439'],
 'Repeat': ['IPR006626']}

In [4]:
# conda activate bio_phylo

import os 
from Bio import SeqIO

#path_db = "/home/conchae/databases/depolymerase_building"
#path_out = "/home/conchae/databases/depolymerase_building/IPR_domain_multi"

path_db = "/home/robbyce/Documents/bioinformatics/depolymerase_building"
path_out = "/home/robbyce/Documents/bioinformatics/depolymerase_building/IPR_domain_multi"


for ipr_multi in os.listdir(f"{path_db}/IPR_multi_fasta"):
    ipr_id = ipr_multi.split(".entry")[0]
    with open(f"{path_out}/{ipr_id}.domain.sequences.fasta", "w") as outfile :
        for record in SeqIO.parse(f"{path_db}/IPR_multi_fasta/{ipr_multi}", "fasta"):
            mid_info = record.id.split("|")[1]
            if mid_info.count(";") > 2 :
                print(ipr_multi)
                break
        
                    
            
            
        
    


IPR027291.entry.sequences.fasta
IPR006215.entry.sequences.fasta
IPR033452.entry.sequences.fasta
IPR001139.entry.sequences.fasta
IPR001554.entry.sequences.fasta
IPR018155.entry.sequences.fasta
IPR002152.entry.sequences.fasta
IPR011330.entry.sequences.fasta
IPR031330.entry.sequences.fasta
IPR018082.entry.sequences.fasta
IPR008291.entry.sequences.fasta
IPR032792.entry.sequences.fasta
IPR001661.entry.sequences.fasta
IPR001362.entry.sequences.fasta
IPR029070.entry.sequences.fasta
IPR005323.entry.sequences.fasta
IPR006103.entry.sequences.fasta
IPR006633.entry.sequences.fasta
IPR015883.entry.sequences.fasta
IPR002241.entry.sequences.fasta


KeyboardInterrupt: 